IMPORT LIBRARY

In [16]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


ADD DATASET

In [17]:
data = pd.read_csv('dirty_cafe_sales.csv')

DATA CHECK

In [18]:
data.head(5)

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [19]:
data.tail(5)

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
9995,TXN_7672686,Coffee,2,2.0,4.0,NaN,UNKNOWN,2023-08-30
9996,TXN_9659401,NaN,3,NaN,3.0,Digital Wallet,NaN,2023-06-02
9997,TXN_5255387,Coffee,4,2.0,8.0,Digital Wallet,NaN,2023-03-02
9998,TXN_7695629,Cookie,3,NaN,3.0,Digital Wallet,NaN,2023-12-02
9999,TXN_6170729,Sandwich,3,4.0,12.0,Cash,In-store,2023-11-07


DATA SUMMARY

In [20]:
data.describe()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
count,10000,9667,9862,9821,9827,7421,6735,9841
unique,10000,10,7,8,19,5,4,367
top,TXN_9226047,Juice,5,3.0,6.0,Digital Wallet,Takeaway,UNKNOWN
freq,1,1171,2013,2429,979,2291,3022,159


In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


DATA TYPE CONVERSION

In [22]:
# Konversi kolom numerik dari tipe objek ke float
data['Quantity'] = pd.to_numeric(data['Quantity'], errors='coerce')
data['Price Per Unit'] = pd.to_numeric(data['Price Per Unit'], errors='coerce')
data['Total Spent'] = pd.to_numeric(data['Total Spent'], errors='coerce')

# Sekarang kolom-kolom tersebut sudah dalam bentuk float dan bisa dipakai untuk analisis statistik
print(data[['Quantity', 'Price Per Unit', 'Total Spent']].dtypes)

Quantity          float64
Price Per Unit    float64
Total Spent       float64
dtype: object


In [23]:
data.describe()

,Quantity,Price Per Unit,Total Spent
count,9521.000000,9467.000000,9498.000000
mean,3.028463,2.949984,8.924352
std,1.419007,1.278450,6.009919
min,1.000000,1.000000,1.000000
25%,2.000000,2.000000,4.000000
50%,3.000000,3.000000,8.000000
75%,4.000000,4.000000,12.000000
max,5.000000,5.000000,25.000000


NULL VALUES HANDLING

In [24]:
# 1. Kolom float yang perlu dibersihkan
float_columns = ['Quantity', 'Price Per Unit', 'Total Spent']
float_invalid_values = ['NaN', 'nan', 'UNKNOWN', 'unknown', 'ERROR', 'Error', 'error', '', ' ', '-', '--']

# 2. Ubah nilai tidak valid jadi NaN
for col in float_columns:
    data[col] = pd.to_numeric(data[col].replace(float_invalid_values, np.nan), errors='coerce')

<ipython-input-24-816cd74f7a03>:31: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(data[col].median(), inplace=True)


In [ ]:
# 3. Mengisi Null Values berdasarkan keterikatan antar kolom
# a. Isi Price Per Unit jika bisa dihitung
mask = data['Price Per Unit'].isnull() & data['Total Spent'].notnull() & data['Quantity'].notnull()
data.loc[mask, 'Price Per Unit'] = data.loc[mask, 'Total Spent'] / data.loc[mask, 'Quantity']

# b. Isi Total Spent jika bisa dihitung
mask = data['Total Spent'].isnull() & data['Quantity'].notnull() & data['Price Per Unit'].notnull()
data.loc[mask, 'Total Spent'] = data.loc[mask, 'Quantity'] * data.loc[mask, 'Price Per Unit']

# c. Isi Quantity jika bisa dihitung
mask = data['Quantity'].isnull() & data['Total Spent'].notnull() & data['Price Per Unit'].notnull()
data.loc[mask, 'Quantity'] = data.loc[mask, 'Total Spent'] / data.loc[mask, 'Price Per Unit']

In [ ]:
# 4. Mengisi sisa Null values nya berdasarkan median per Item
for col in float_columns:
    missing_mask = data[col].isnull() & data['Item'].notnull()
    median_per_item = data.groupby('Item')[col].median()
    data.loc[missing_mask, col] = data.loc[missing_mask, 'Item'].map(median_per_item)

In [ ]:
# 5. Mengisi Null Values terakhir dengan median global
for col in float_columns:
    if data[col].isnull().sum() > 0:
        data[col].fillna(data[col].median(), inplace=True)

In [25]:
# Daftar kolom tipe objek yang mau dibersihkan
object_columns = ['Item', 'Payment Method', 'Location']

# Nilai-nilai yang dianggap "kosong" secara makna
invalid_values = ['NaN', 'nan', 'UNKNOWN', 'unknown', '', ' ', '-', '--']

for col in object_columns:
    # Ganti nilai error jadi np.nan
    data[col] = data[col].replace(invalid_values, np.nan)

    # Isi NaN dengan modus kolom tersebut
    if data[col].isnull().sum() > 0:
        mode_val = data[col].mode().iloc[0]
        data[col] = data[col].fillna(mode_val)

In [26]:
# Konversi ke datetime
data['Transaction Date'] = pd.to_datetime(data['Transaction Date'], errors='coerce')

# Isi yang kosong dengan median datetime (opsional)
median_date = data['Transaction Date'].median()
data['Transaction Date'] = data['Transaction Date'].fillna(median_date)


In [27]:
data.isna().sum()

,0
Transaction ID,0
Item,0
Quantity,0
Price Per Unit,0
Total Spent,0
Payment Method,0
Location,0
Transaction Date,0


In [30]:
data.sample(30)

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
1805,TXN_5113993,Coffee,1.0,2.0,2.0,Credit Card,Takeaway,2023-02-10
8314,TXN_8082317,Juice,3.0,3.0,9.0,Cash,Takeaway,2023-08-18
1938,TXN_1643039,Cookie,3.0,1.0,3.0,Digital Wallet,Takeaway,2023-02-17
5000,TXN_2756336,Smoothie,3.0,4.0,12.0,Credit Card,Takeaway,2023-07-07
9083,TXN_1528454,Coffee,2.0,2.0,4.0,Digital Wallet,In-store,2023-03-22
9570,TXN_3125699,Juice,1.0,3.0,3.0,Credit Card,Takeaway,2023-02-05
7947,TXN_3521791,Coffee,1.0,2.0,2.0,Digital Wallet,Takeaway,2023-06-11
3677,TXN_9489442,Juice,3.0,2.0,6.0,Credit Card,Takeaway,2023-02-06
4868,TXN_4956012,Sandwich,1.0,4.0,4.0,Digital Wallet,In-store,2023-07-20
9745,TXN_6650875,Cookie,5.0,1.0,5.0,Credit Card,Takeaway,2023-01-29


DUPLICATE CHECK

In [33]:
# Mengecek apakah ada duplicate di seluruh kolom
check_duplicate = data.duplicated().sum()

print(f"Jumlah data yang duplikat = {check_duplicate}")

Jumlah data yang duplikat = 0
